In [9]:
import requests
import base64
import os
import io
from io import BytesIO

def upload_image(imagebase64str: str) -> str:
    image = base64.b64decode(imagebase64str)

    # Convert to a bytes-like object for the requests library
    image_file = BytesIO(image)
    image_file.name = 'image.jpg'  # You can specify the filename here
    image_file.seek(0)  # Move to the start of the file

    files = {'file': (image_file.name, image_file, 'image/jpeg')}
    response = requests.post(f"http://127.0.0.1:8002/images/", files=files)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error uploading image: {response.status_code}, {response.text}")

def detect_activity_inside_video(video_id) -> dict:
    response = requests.post(f"http://127.0.0.1:16004/video-inference-with-id/{video_id}")
    if response.status_code == 200:
        json_file = response.json()
        json_file['type'] = 'ecole-video-activity'
        json_file['data'] = f"The video depicts the activity of '{json_file['pred']}'"
        json_file['merged_image'] = upload_image(json_file['merged_image'])
        del json_file['query_prompt']
        return json_file
    else:
        raise Exception(f"Error fetching video: {response.status_code}, {response.text}")

In [10]:
detect_activity_inside_video("65c5f6aaf09b5ac909575a60")

Exception: Error uploading image: 500, Internal Server Error